In [172]:
import nltk

import io 
import codecs
import csv
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk import stem

import re
import pandas as pd

from nltk.corpus import stopwords
from string import punctuation
import ast
from collections import Counter

import heapq
import math

In [42]:
def cleaner(text):

    words = re.split(r'\W+', str(text))
    words = [word.lower() for word in words]
    without_punct = [wp for wp in words if wp not in punctuation]
    sw = stopwords.words('english')
    without_sw = [w for w in without_punct if w not in sw] 
    list_to_remove = ['b','br','span', 'one' , 'id', 'none' ]
    clean_more = [w for w in without_sw if w not in list_to_remove] 
    clean_more_2 = [w for w in clean_more if  not (re.findall(re.compile(r'freetext'),w)
                                                   or re.findall(re.compile(r'\d'),w)) ]
    
    
    ps = nltk.stem.PorterStemmer()
    stemmed_list=[ps.stem(w) for w in clean_more_2 ]
    return ' '.join(stemmed_list)

In [64]:
for books in range(20001,30001):
    text_1=''
    try:
        df = pd.read_csv("books_3_tsv/article_"+str(books)+".tsv", sep='\t', encoding='utf-8' ) #vocab read csv files
    except:
        continue
    df = df.fillna("")
    df['book_title']=df['book_title'].apply(cleaner)
    df['complete_plot']=df['complete_plot'].apply(cleaner)
    #df['book_series']=df['book_series'].apply(cleaner)

    #df.to_csv('cleaned_csv/prova_libro_'+str(books)+'.csv') #to create new cleaned csv use this line 
    #now create the vocab
    
    title = df.iloc[0, 0]
    plot = df.iloc[0, 8]
    text_1 = title + " " + plot
    arr_1=(text_1.split())
    c = Counter(arr_1)
    file = open("txt_files_2/article_"+str(books)+".txt", "w") 
    file.write(str(c)) 
    file.close() 

 #save vocabulary as csv with indices for each vocab

In [57]:
file = open("txt_files_2/article_"+str(20300)+".txt", "r") 
dic=file.read() 
file.close() 

In [53]:
dictio=ast.literal_eval(dic)

In [61]:
prova=dic.replace('Counter(','').replace(')','')

In [127]:
from collections import defaultdict
d = defaultdict(list)

In [69]:
def extract_from_counter(id):
    try:
        
        file = open("txt_files_2/article_"+str(doc)+ ".txt", "r")
        dic=file.read()
        prova=dic.replace('Counter(','').replace(')','')
        test=ast.literal_eval(prova)
        file.close()
        return test
    except:
        return {}

In [128]:
for doc in range(20000,30001):
    result=extract_from_counter(doc)
    for key in result.keys():
        heapq.heappush(d[key],(result[key],doc))
        #d[key].append((doc,result[key]))  #substitute with heap push

In [95]:
df_voc= pd.read_csv('vocab.csv')
df_voc.columns=['id','word']

In [132]:
dic_keys=d.keys()
new_dic=defaultdict(list)
for item in dic_keys:
    try:
        new_key=df_voc[df_voc.word==item].id.iloc[0]
        new_dic[new_key]=d[item]
    except:
        continue

In [136]:
new_dic[123]

[(1, 24068), (2, 21796)]

In [148]:
#save the lenght of each document in a dictionary
dic_lenght=defaultdict(int)

In [149]:
for doc in range(20001,30001):
    result=extract_from_counter(doc)
    lenght=sum(result.values())
    if(lenght==0):
        continue
    dic_lenght[doc]=lenght

In [150]:
dic_lenght # i will use this for the denominator of tf

defaultdict(int,
            {20001: 7,
             20002: 118,
             20004: 133,
             20005: 167,
             20006: 143,
             20007: 110,
             20008: 118,
             20009: 88,
             20010: 131,
             20011: 174,
             20012: 90,
             20014: 2,
             20015: 87,
             20016: 23,
             20017: 114,
             20018: 166,
             20021: 126,
             20022: 32,
             20023: 187,
             20025: 143,
             20027: 101,
             20028: 188,
             20030: 144,
             20031: 173,
             20032: 131,
             20034: 95,
             20035: 131,
             20036: 147,
             20037: 130,
             20038: 276,
             20039: 103,
             20040: 108,
             20041: 10,
             20042: 176,
             20043: 169,
             20044: 88,
             20045: 2,
             20046: 101,
             20047: 106,
             20049: 76

In [158]:
documents_number=10000

In [170]:
len(d['jack'])

115

In [166]:
# save the number of documents for each word in a different dictionary
documents_with_word= defaultdict(int)
for item in d.keys():
    documents_with_word[item]=len(d[item])

    

In [173]:
#create again the dictionary with the heap ordered by tf_idf
final_inverted_index=defaultdict(list)
for doc in range(20000,30001):
    result=extract_from_counter(doc)
    doc_len=dic_lenght[doc]
    for key in result.keys():
        tf= result[key] / doc_len
        idf= math.log(documents_number / documents_with_word[key] ,10)
        tf_idf= tf * idf
        heapq.heappush(final_inverted_index[key],(tf_idf,doc))

In [178]:
#change the word with the numbers 
dic_keys=final_inverted_index.keys()
inverted_index_tfidf=defaultdict(list)
for item in dic_keys:
    try:
        new_key=df_voc[df_voc.word==item].id.iloc[0]
        inverted_index_tfidf[new_key]=final_inverted_index[item]
    except:
        continue

In [180]:
inverted_index_tfidf[0]

[(0.011948205202325108, 23355),
 (0.014049648328362186, 21207),
 (0.014561875090333723, 22014),
 (0.026883461705231492, 21997),
 (0.01419228435200038, 23642),
 (0.017152638143215184, 25650),
 (0.05325485747322048, 28295),
 (0.0380391839094432, 21155),
 (0.04142044470139371, 23490),
 (0.033483593022084736, 21765),
 (0.046212892848662396, 24129),
 (0.03584461560697532, 24395),
 (0.0317713638334554, 20422),
 (0.07820643405158252, 22309),
 (0.06819219554497744, 28666),
 (0.03965787258644078, 29073)]